### Building a RAG System with LangChain and ChromaDB

#### Introduction 
RAG is a technique with which we increase the answering capabilities of LLM by giving them 
relevant, updated context from external source utilizing the language and knowledge acquired by the LLMS through training

* LangChain : A framework which is useful in developing applications powered by language models
* ChromaDB : A vector database which we can use to store and search for vector embeddings to get the relevant context
* OpenAI : we can get pretrained LLM models and embedding models

In [42]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [43]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

## vectorstores 
from langchain_community.vectorstores import Chroma

In [44]:
print("""
RAG (Retrieval-Augmented Generation) Architecture:

1. Document Loading: Load documents from various sources
2. Document Splitting: Break documents into smaller chunks
3. Embedding Generation: Convert chunks into vector representations
4. Vector Storage: Store embeddings in ChromaDB
5. Query Processing: Convert user query to embedding
6. Similarity Search: Find relevant chunks from vector store
7. Context Augmentation: Combine retrieved chunks with query
8. Response Generation: LLM generates answer using context

Benefits of RAG:
- Reduces hallucinations
- Provides up-to-date information
- Allows citing sources
- Works with domain-specific knowledge
""")


RAG (Retrieval-Augmented Generation) Architecture:

1. Document Loading: Load documents from various sources
2. Document Splitting: Break documents into smaller chunks
3. Embedding Generation: Convert chunks into vector representations
4. Vector Storage: Store embeddings in ChromaDB
5. Query Processing: Convert user query to embedding
6. Similarity Search: Find relevant chunks from vector store
7. Context Augmentation: Combine retrieved chunks with query
8. Response Generation: LLM generates answer using context

Benefits of RAG:
- Reduces hallucinations
- Provides up-to-date information
- Allows citing sources
- Works with domain-specific knowledge



#### 1.Data Gathering

In [183]:
## create sample documents
sample_docs = [
    """
    Machine Learning Fundamentals
    
    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties.
    """,
    
    """
    Deep Learning and Neural Networks
    
    Deep learning is a subset of machine learning based on artificial neural networks. 
    These networks are inspired by the human brain and consist of layers of interconnected 
    nodes. Deep learning has revolutionized fields like computer vision, natural language 
    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly 
    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers 
    excel at sequential data processing.
    """,
    
    """
    Natural Language Processing (NLP)
    
    NLP is a field of AI that focuses on the interaction between computers and human language. 
    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, 
    machine translation, and question answering. Modern NLP heavily relies on transformer 
    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand 
    context and relationships between words in text.
    """
]

sample_docs


['\n    Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through \n    interaction with an environment using rewards and penalties.\n    ',
 '\n    Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly \n    effective f

In [184]:
## save files temporarily
import tempfile 

temp_dir = tempfile.mkdtemp()

for i, content in enumerate(sample_docs):
    with open(f"{temp_dir}/doc_{i+1}.txt", 'w') as empty_file:
        empty_file.write(content)

print(f"Directory: {temp_dir}")
print(*list(os.listdir(temp_dir)))

Directory: C:\Users\rohit\AppData\Local\Temp\tmpxjwsaa9p
doc_1.txt doc_2.txt doc_3.txt


#### 2. Document Loading

In [17]:
temp_dir = "C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p"

In [18]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader

## load all the text files from the directory 
loader = DirectoryLoader(
    temp_dir, 
    glob = ['**/doc_*.txt'], 
    loader_cls = TextLoader,
    loader_kwargs = {'encoding': 'utf-8'}
)

documents = loader.load()
print(f"Total Loaded Documents: {len(documents)}")
print(f"======")
print(f"\n First document preview: \n {documents[0].page_content}")
print(f"Metadata: {documents[0].metadata}")

Total Loaded Documents: 3

 First document preview: 
 
    Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties.
    
Metadata: {'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_1.txt'}


In [19]:
repr(sample_docs[0])

NameError: name 'sample_docs' is not defined

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [21]:
### Document Splitting 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, # would like to experiment by lowering this
    chunk_overlap=50,
    length_function=len,
    separators=[" "] 

)

chunks = text_splitter.split_documents(documents)

print(f"Created {len(chunks)} chunks from {len(documents)} documents")
print(f"=============")
print(f"\nChunk exmaple:")
print(f"Content: {chunks[0].page_content[: 150]}...")
print(f"Metadata: {chunks[0].metadata}")

Created 5 chunks from 3 documents

Chunk exmaple:
Content: Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experie...
Metadata: {'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_1.txt'}


#### Embedding Models

#### Initializing ChromaDB Vector Store

In [26]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [27]:
vdb_directory = "./chroma.db"

## Initialize Chromadb  with OpenAI embeddings
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    persist_directory=vdb_directory,
    collection_name="rag_vdb_collection"
)

In [28]:
for att_or_met in dir(vectorstore):
    if not att_or_met.startswith("_"):
        print(att_or_met)

aadd_documents
aadd_texts
add_documents
add_images
add_texts
adelete
afrom_documents
afrom_texts
aget_by_ids
amax_marginal_relevance_search
amax_marginal_relevance_search_by_vector
as_retriever
asearch
asimilarity_search
asimilarity_search_by_vector
asimilarity_search_with_relevance_scores
asimilarity_search_with_score
delete
delete_collection
embeddings
encode_image
from_documents
from_texts
get
get_by_ids
max_marginal_relevance_search
max_marginal_relevance_search_by_vector
override_relevance_score_fn
persist
search
similarity_search
similarity_search_by_image
similarity_search_by_image_with_relevance_score
similarity_search_by_vector
similarity_search_by_vector_with_relevance_scores
similarity_search_with_relevance_scores
similarity_search_with_score
update_document
update_documents


#### Text-Similarity Search

In [29]:
query = "What are the types of machine learning ?"

similar_docs=vectorstore.similarity_search(query, k=3)
similar_docs

[Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_1.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
 Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_1.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learni

In [30]:
def query_search(query, vector_store=vectorstore, top_k=3):
    return vector_store.similarity_search(query, k=top_k)

In [31]:
query_search("what is NLP?")

[Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_3.txt'}, page_content='Natural Language Processing (NLP)\n\n    NLP is a field of AI that focuses on the interaction between computers and human language. \n    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, \n    machine translation, and question answering. Modern NLP heavily relies on transformer \n    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand \n    context and relationships between words in text.'),
 Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_3.txt'}, page_content='Natural Language Processing (NLP)\n\n    NLP is a field of AI that focuses on the interaction between computers and human language. \n    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, \n    machine translation, and question answering. Modern NLP heavily re

In [32]:
query_search("What is Deep Learning?")

[Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly \n    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers'),
 Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like c

In [33]:
def print_query_results(query, vector_store=vectorstore, top_k=3):
    print(f"for query: {query}")
    print(f"----------------------")
    print(f"The {top_k} similar results are: ")
    print(f"---------")
    for matching in query_search(query, vector_store=vector_store, top_k=top_k):
        print(f"Source: {matching.metadata}")
        print(f"Content: {matching.page_content}")
        print(f"\n\n")

In [34]:
print_query_results("What is machine Learning", top_k=1)

for query: What is machine Learning
----------------------
The 1 similar results are: 
---------
Source: {'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_1.txt'}
Content: Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through





#### Initialize LLM, RAG Chain, Prompt Template, Query RAG

In [35]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(
    model_name="gpt-3.5-turbo"
)

In [197]:
test_response = llm.invoke("what is LLM in AI?")
test_response

AIMessage(content='LLM stands for Language Model for Large-scale semi-structured data. This is a deep learning model developed for processing and understanding large amounts of unstructured or semi-structured data, such as text and speech. It uses artificial intelligence techniques to analyze, interpret, and derive insights from this data, enabling applications such as natural language processing, machine translation, sentiment analysis, and more.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 14, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cctn9KrWMqmRilpAqX5TzWnyi0HST', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs'

In [198]:
test_response.content

'LLM stands for Language Model for Large-scale semi-structured data. This is a deep learning model developed for processing and understanding large amounts of unstructured or semi-structured data, such as text and speech. It uses artificial intelligence techniques to analyze, interpret, and derive insights from this data, enabling applications such as natural language processing, machine translation, sentiment analysis, and more.'

In [199]:
test_response

AIMessage(content='LLM stands for Language Model for Large-scale semi-structured data. This is a deep learning model developed for processing and understanding large amounts of unstructured or semi-structured data, such as text and speech. It uses artificial intelligence techniques to analyze, interpret, and derive insights from this data, enabling applications such as natural language processing, machine translation, sentiment analysis, and more.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 14, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cctn9KrWMqmRilpAqX5TzWnyi0HST', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs'

In [200]:
from langchain.chat_models.base import init_chat_model

llm=init_chat_model("openai:gpt-3.5-turbo")

In [201]:
llm.invoke("Who is the president of America?")

AIMessage(content='As of September 2021, the President of the United States is Joseph R. Biden Jr.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 14, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CctnFC1kuIfb4kYJJQerNgrXj9lTy', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--4695dbbe-501e-426b-afff-bc8c1f9fa109-0', usage_metadata={'input_tokens': 14, 'output_tokens': 20, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [202]:
print(llm.invoke("Who is the president of America?").content)
print(llm.invoke("Who is the chief minister of Telanga State, India?").content)

As of October 2021, the president of America is Joe Biden.
As of September 2021, K. Chandrashekar Rao is the Chief Minister of Telangana State, India.


In [203]:
## These are the examples which shows how we get wrong answers which were correct 
## at the training time of the data but is oudated info and incorrect currently 


#### Modern RAG chain

In [204]:
from langchain_core.runnables import RunnableSequence
from langchain_core.prompts import PromptTemplate

In [205]:
# dir(MultiQueryRetriever())
# will be useful when have to generate prompts 
# from query_prompt for vector search in retreiver. 
# all the necessary context 

In [206]:
retriever = vectorstore.as_retriever(search_kwargs= {"k": 3})
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002B9C97612B0>, search_kwargs={'k': 3})

In [207]:
retriever.invoke("What is Machine Learning?")

[Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_1.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
 Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer visi

In [208]:
system_prompt="""You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
the context is after Content:  and the input or the question is 
after Input:

Context: {context}
Input: {Input}
"""

In [209]:
prompt_template = PromptTemplate(
    template= system_prompt

)

prompt_template

PromptTemplate(input_variables=['Input', 'context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nthe context is after Content:  and the input or the question is \nafter Input:\n\nContext: {context}\nInput: {Input}\n")

In [210]:
prompt_template.invoke({"context": "Modi is Great?", "Input": "who is great?"})

StringPromptValue(text="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nthe context is after Content:  and the input or the question is \nafter Input:\n\nContext: Modi is Great?\nInput: who is great?\n")

In [43]:
def join_docs(docs):
    combined_content = "\n".join([doc.page_content for doc in docs])
    return combined_content

In [212]:
from langchain_core.runnables import chain

In [44]:
# gives us a runnable with the same name as that of the function which is given to us by the chain decorator.
@chain
def get_response_with_context(qyery, vector_store=vectorstore,
                              prompt_template=prompt_template, llm=llm, top_k=3):
    similar_docs = vector_store.similarity_search(query, k=top_k)
    context = join_docs(similar_docs)

    prompt = prompt_template.invoke({"context": context, "Input": query})
    
    response = llm.invoke(prompt)
    
    return response.content

In [45]:
join_docs(vectorstore.similarity_search("What is Machine Learning?", k=3))

'Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through\nMachine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through\nDeep Learning and Neura

In [215]:
get_response_with_context.invoke("What is Machine Learning?")

'Supervised learning, unsupervised learning, and reinforcement learning are the three main types of machine learning. Supervised learning uses labeled data, unsupervised learning finds patterns in unlabeled data, and reinforcement learning learns through feedback from interactions in an environment.'

In [216]:
# ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate

In [217]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

prompt_value = template.invoke(
    {
        "name": "Bob",
        "user_input": "What is your name?"
    }
)

In [218]:
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})])

In [219]:
llm.invoke(prompt_value)

AIMessage(content='My name is Bob. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 50, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CctnTLE9AKZtwCGuKplZxeOQjRlFA', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--a2c81fde-f87c-4aac-a5e6-f92dbbc82fd8-0', usage_metadata={'input_tokens': 50, 'output_tokens': 12, 'total_tokens': 62, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [220]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

In [221]:
from langchain_core.runnables.passthrough import (RunnableAssign,
                                                   RunnableParallel, 
                                                   RunnablePassthrough,
                                                   )

In [46]:
def get_context(query, vector_store=vectorstore, top_k=3):
    similar_docs = vector_store.similarity_search(query, k=top_k)
    context = join_docs(similar_docs)
    return context

In [223]:
to_prompt_template = RunnableParallel( 
    context = RunnableLambda(get_context),
    Input = RunnablePassthrough()
)

In [224]:
llm_pipeline = RunnableSequence(to_prompt_template, prompt_template, llm)

In [225]:
llm_pipeline.invoke("What is Machine Learning?")

AIMessage(content='Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It includes supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data, unsupervised learning finds patterns in unlabeled data, and reinforcement learning learns through interaction with an environment to achieve a goal.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 355, 'total_tokens': 423, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CctnVEMjOYH4FQgKo13727xMP4VRb', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None},

In [ ]:
# pipeline = retriever | prompt_template | llm

In [88]:
# "\n\n".join([doc.page_content for doc in retriever.invoke("what is machine learning?")])

In [89]:
# out = pipeline.invoke({"Input": "how are  you?"})
# the above line doesn't work since the input invoke on the retriever can't deal 
# dictionary input and convert it into a query string.

In [47]:
def get_context_dict(query: dict, vector_store=vectorstore, top_k=3):
    similar_docs = vector_store.similarity_search(query["Input"], k=top_k)
    context = join_docs(similar_docs)
    return context

In [227]:
template_input = RunnableAssign(
    RunnableParallel(
        context= RunnableLambda(get_context_dict)
    )
)

In [228]:
query_call = template_input | prompt_template | llm

In [229]:
query_call.invoke({"Input": "What is Machine Learning?"})

AIMessage(content='Machine learning is a subset of artificial intelligence that enables systems to learn from experience. There are three main types: supervised, unsupervised, and reinforcement learning. Supervised learning uses labeled data to train models.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 355, 'total_tokens': 397, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CctndNdCZnsWC31YDLGVhSTaaIX3L', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--7135e4d4-0828-4d5c-bcab-600c79bf854b-0', usage_metadata={'input_tokens': 355, 'output_tokens': 42, 'total_tokens': 397, 'input_token_details': {'audio': 0, 'cache_read': 

In [230]:
answer = query_call.invoke({"Input": "What is Deep Learning?"})

In [231]:
answer.content

'Deep learning is a subset of machine learning that uses artificial neural networks inspired by the human brain. It has revolutionized fields like computer vision, natural language processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly effective for image processing.'

In [232]:
only_answer_output = template_input | prompt_template | llm | (lambda x: x.content)

In [233]:
only_answer_output.invoke({"Input": "What is Natural Language Processing?"})

'Natural Language Processing (NLP) is a field of AI focused on computers interacting with human language. Key tasks in NLP include text classification and sentiment analysis. NLP heavily relies on transformer architectures like BERT and GPT.'

In [234]:
answer_to_query = RunnableLambda(
    lambda query: {"Input": query}
).assign(docs = lambda out: retriever.invoke(out["Input"])
).assign(context = lambda out: join_docs(out["docs"])
).assign(prompt = lambda out: prompt_template.invoke({"Input": out["Input"], "context": out["context"]})
         ).assign(response = lambda out: llm.invoke(out["prompt"])
         ).assign(answer = lambda out: out["response"].content)

In [235]:
output = answer_to_query.invoke("What is Reinforcement Learning?")

In [236]:
output["answer"]

'Reinforcement learning learns through interaction with an environment, using rewards and penalties for feedback. It is a type of machine learning that falls under the category of artificial intelligence. This method allows systems to learn and improve based on their experiences without explicit programming.'

In [237]:
dir(output["response"])

['__abstractmethods__',
 '__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_on_complete__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_

In [238]:
output["response"].response_metadata

{'token_usage': {'completion_tokens': 50,
  'prompt_tokens': 283,
  'total_tokens': 333,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_provider': 'openai',
 'model_name': 'gpt-3.5-turbo-0125',
 'system_fingerprint': None,
 'id': 'chatcmpl-CctnlH0zTtkXk6NBj2wsKOJ0hL2zU',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

In [239]:
output["docs"]

[Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_1.txt'}, page_content='data. Reinforcement learning learns through \n    interaction with an environment using rewards and penalties.'),
 Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_1.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
 Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a sub

In [240]:
output["context"]

'data. Reinforcement learning learns through \n    interaction with an environment using rewards and penalties.\nMachine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through\nDeep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particu

In [241]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [242]:
# create a custom prompt
custom_prompt = ChatPromptTemplate.from_template("""Use the following context to answer the question.
If you don't know the answer, just say that you don't know.
Provide specific details from the context to support your answer.

Context: {context}

Question :{question}

Answer:""")

custom_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following context to answer the question.\nIf you don't know the answer, just say that you don't know.\nProvide specific details from the context to support your answer.\n\nContext: {context}\n\nQuestion :{question}\n\nAnswer:"), additional_kwargs={})])

In [243]:
## Format the output documents for the prompt 
def format_docs(docs):
    return "\n\n".join([f"Content: {doc.page_content}" for doc in docs])

In [244]:
## Build a chain using LCEL 

rag_chain_lcel = ({"context": retriever | format_docs,
                   "question": RunnablePassthrough()
                  }
                  | custom_prompt
                  | llm
                  | StrOutputParser()
                 )

In [245]:
rag_chain_lcel.invoke("What do you think about Deep learning?")

'Deep learning is a subset of machine learning that is based on artificial neural networks. It has revolutionized fields such as computer vision, natural language processing, and speech recognition. Convolutional Neural Networks are particularly effective for image processing, while Recurrent Neural Networks and Transformers excel at sequential data processing.'

In [246]:
retriever.invoke("What do you think about Deep learning?")

[Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly \n    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers'),
 Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_1.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsuper

In [247]:
def query_rag_lcel(question):
    print(f"Question: {question}")
    print("-" * 40)

    answer = rag_chain_lcel.invoke(question)
    print(f"Answer: {answer}")

    # Get Source documents seperately if needed 
    print("\nSource Documents:")
    for i, doc in enumerate(retriever.invoke(question)):
        print(f"\n----Source Document {i+1}:")
        print(f"Content: {doc.page_content}")
        print(f"Metadata: {doc.metadata}")
        print("\n")

In [248]:
query_rag_lcel("What is Natural Language Processing?")

Question: What is Natural Language Processing?
----------------------------------------
Answer: Natural Language Processing (NLP) is a field of AI that focuses on the interaction between computers and human language. Key tasks in NLP include text classification, named entity recognition, sentiment analysis, machine translation, and question answering. NLP heavily relies on transformer architectures like BERT, GPT, and T5, which use attention mechanisms to understand context and relationships between words in text.

Source Documents:

----Source Document 1:
Content: Natural Language Processing (NLP)

    NLP is a field of AI that focuses on the interaction between computers and human language. 
    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, 
    machine translation, and question answering. Modern NLP heavily relies on transformer 
    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand 
    context an

In [249]:
new_file_content = """
This is a newly added file for testing. 
It discusses the applications of Artificial Intelligence (AI) in various industries.
AI has transformed sectors such as healthcare, finance, and transportation. 
Machine learning and deep learning are key components of AI that enable systems to learn from data and make predictions.
"""

In [250]:
vectorstore.add_documents([Document(page_content=new_file_content, metadata={"source": "new_doc.txt"})])

['b457dcbb-040d-45e4-8cc5-c4607c5e802a']

In [253]:
vectorstore.delete(['b457dcbb-040d-45e4-8cc5-c4607c5e802a'])

In [251]:
new_document = """
Reinforcement Learning in Detail

Reinforcement learning (RL) is a type of machine learning where an agent learns to make 
decisions by interacting with an environment. The agent receives rewards or penalties 
based on its actions and learns to maximize cumulative reward over time. Key concepts 
in RL include: states, actions, rewards, policies, and value functions. Popular RL 
algorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and 
Actor-Critic methods. RL has been successfully applied to game playing (like AlphaGo), 
robotics, and autonomous systems.
"""


In [252]:
new_document

'\nReinforcement Learning in Detail\n\nReinforcement learning (RL) is a type of machine learning where an agent learns to make \ndecisions by interacting with an environment. The agent receives rewards or penalties \nbased on its actions and learns to maximize cumulative reward over time. Key concepts \nin RL include: states, actions, rewards, policies, and value functions. Popular RL \nalgorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and \nActor-Critic methods. RL has been successfully applied to game playing (like AlphaGo), \nrobotics, and autonomous systems.\n'

In [254]:
new_doc = Document(
    page_content=new_document,
    metadata= {"source": "manual_addition",
    "topic": "Reinforcement Learning"}
)

new_doc

Document(metadata={'source': 'manual_addition', 'topic': 'Reinforcement Learning'}, page_content='\nReinforcement Learning in Detail\n\nReinforcement learning (RL) is a type of machine learning where an agent learns to make \ndecisions by interacting with an environment. The agent receives rewards or penalties \nbased on its actions and learns to maximize cumulative reward over time. Key concepts \nin RL include: states, actions, rewards, policies, and value functions. Popular RL \nalgorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and \nActor-Critic methods. RL has been successfully applied to game playing (like AlphaGo), \nrobotics, and autonomous systems.\n')

In [255]:
new_chunks = text_splitter.split_documents([new_doc])
new_chunks

[Document(metadata={'source': 'manual_addition', 'topic': 'Reinforcement Learning'}, page_content='Reinforcement Learning in Detail\n\nReinforcement learning (RL) is a type of machine learning where an agent learns to make \ndecisions by interacting with an environment. The agent receives rewards or penalties \nbased on its actions and learns to maximize cumulative reward over time. Key concepts \nin RL include: states, actions, rewards, policies, and value functions. Popular RL \nalgorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and \nActor-Critic methods. RL has been'),
 Document(metadata={'source': 'manual_addition', 'topic': 'Reinforcement Learning'}, page_content='methods, and \nActor-Critic methods. RL has been successfully applied to game playing (like AlphaGo), \nrobotics, and autonomous systems.')]

In [256]:
### Add new document to the vector store 
vectorstore.add_documents(new_chunks)

['f300f1af-410e-45a9-85d2-085eb66e753c',
 '497a198f-325f-4249-951d-2372005c9b22']

In [257]:
vectorstore.similarity_search("Advanced Reinforcement Learning", k=3)

[Document(metadata={'topic': 'Reinforcement Learning', 'source': 'manual_addition'}, page_content='Reinforcement Learning in Detail\n\nReinforcement learning (RL) is a type of machine learning where an agent learns to make \ndecisions by interacting with an environment. The agent receives rewards or penalties \nbased on its actions and learns to maximize cumulative reward over time. Key concepts \nin RL include: states, actions, rewards, policies, and value functions. Popular RL \nalgorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and \nActor-Critic methods. RL has been'),
 Document(metadata={'source': 'C:\\Users\\rohit\\AppData\\Local\\Temp\\tmpxjwsaa9p\\doc_1.txt'}, page_content='data. Reinforcement learning learns through \n    interaction with an environment using rewards and penalties.'),
 Document(metadata={'topic': 'Reinforcement Learning', 'source': 'manual_addition'}, page_content='methods, and \nActor-Critic methods. RL has been successfully applie

In [179]:
dir(vectorstore)

['_Chroma__query_collection',
 '_LANGCHAIN_DEFAULT_COLLECTION_NAME',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_asimilarity_search_with_relevance_scores',
 '_client',
 '_client_settings',
 '_collection',
 '_cosine_relevance_score_fn',
 '_embedding_function',
 '_euclidean_relevance_score_fn',
 '_get_retriever_tags',
 '_max_inner_product_relevance_score_fn',
 '_persist_directory',
 '_select_relevance_score_fn',
 '_similarity_search_with_relevance_scores',
 'aadd_documents',
 'aadd_texts',
 'add_documents',
 'add_images',
 'add_texts',
 'adelete',
 'afrom_documents'

In [181]:
vectorstore.delete_collection()

In [262]:
def program_end():
    import shutil
    shutil.rmtree(temp_dir, ignore_errors=True)

In [264]:
new_question = "What are the key concepts in Reinforcement Learning?"
result = query_rag_lcel(new_question)

Question: What are the key concepts in Reinforcement Learning?
----------------------------------------
Answer: The key concepts in Reinforcement Learning are states, actions, rewards, policies, and value functions.

Source Documents:

----Source Document 1:
Content: Reinforcement Learning in Detail

Reinforcement learning (RL) is a type of machine learning where an agent learns to make 
decisions by interacting with an environment. The agent receives rewards or penalties 
based on its actions and learns to maximize cumulative reward over time. Key concepts 
in RL include: states, actions, rewards, policies, and value functions. Popular RL 
algorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and 
Actor-Critic methods. RL has been
Metadata: {'source': 'manual_addition', 'topic': 'Reinforcement Learning'}



----Source Document 2:
Content: data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties.
Metadata: {'s

In [275]:
from langchain_core.runnables import RunnableMap

In [276]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.runnables import RunnableLambda, RunnableSequence, RunnableParallel

In [12]:
chat_history = []

In [38]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
        You like answering questions without hallucinating.
        You use the provided context and previous chat history to answer the question in a 
     conversational manner and keep the answers concise. 

     Context: {context}
     """), 
    MessagesPlaceholder(variable_name="history"),
    ("human", "{query}")])


In [39]:
prompt_template.invoke({"history": [HumanMessage("Hi !!")], "context": Document(page_content="for now empty context"),
                        "query": "What is Machine Learning?"})

ChatPromptValue(messages=[SystemMessage(content="\n        You like answering questions without hallucinating.\n        You use the provided context and previous chat history to answer the question in a \n     conversational manner and keep the answers concise. \n\n     Context: page_content='for now empty context'\n     ", additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi !!', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is Machine Learning?', additional_kwargs={}, response_metadata={})])

In [40]:
from langchain_core.runnables import chain

In [48]:
@chain
def conversation_chain(query, chat_history=chat_history,
                       vector_store=vectorstore,
                       prompt_template=prompt_template,
                       llm=llm, top_k=3):
    similar_docs = vector_store.similarity_search(query, k=top_k)
    context = join_docs(similar_docs)

    prompt = prompt_template.invoke({
        "context": Document(page_content=context),
        "history": chat_history,
        "query": query
    })
    
    response = llm.invoke(prompt)
    
    # Update chat history
    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=response.content))
    
    return response.content

In [49]:
conversation_chain.invoke("What is Machine Learning?")

'Machine learning is a subset of artificial intelligence that allows systems to learn and improve from experience without being explicitly programmed. It involves training models on data to make predictions or decisions. There are three main types of machine learning: supervised, unsupervised, and reinforcement learning. Each type has its own approach to learning from data.'

In [50]:
conversation_chain.invoke("can you give me your previous answer once again?")

"Sure! Here's the answer: \nMachine learning is a subset of artificial intelligence that allows systems to learn and improve from experience without being explicitly programmed. It involves training models on data to make predictions or decisions. There are three main types of machine learning: supervised, unsupervised, and reinforcement learning. Each type has its own approach to learning from data."

In [51]:
conversation_chain.invoke("So can you get into more details for your answer")

'In machine learning, algorithms learn from data to make predictions or decisions without being explicitly programmed. \n\n1. Supervised learning involves training models on labeled data, where the algorithm learns patterns to make predictions.\n2. Unsupervised learning finds patterns in data without predefined labels, useful for clustering or dimensionality reduction.\n3. Reinforcement learning learns through trial and error, receiving feedback in the form of rewards or penalties to optimize decisions.\n\nThese approaches enable systems to improve performance over time and adapt to new data without human intervention.'

In [52]:
llm =ChatOpenAI(
    model_name="gpt-4-turbo"
    )

In [53]:
conversation_chain.invoke("So what are the types of machine learning there?")

'The three main types of machine learning are:\n1. Supervised learning: Uses labeled data to train models and make predictions.\n2. Unsupervised learning: Finds patterns in unlabeled data to discover hidden insights.\n3. Reinforcement learning: Learns through trial and error by receiving feedback in the form of rewards or penalties to optimize decisions.'

In [54]:
## LCEL approach for conversation chain
from langchain_core.runnables import RunnableAssign, RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [55]:
prompt_template.input_variables

['context', 'history', 'query']

In [ ]:
def query_dict(query): 
    return {"query": query}


def answer_content(out):
    answer = out["response"].content
    query = out["query"]
    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=answer))
    return answer

In [57]:
my_lcel_try = RunnableLambda(
    query_dict
).assign(history = lambda out: chat_history
).assign(context = lambda out: join_docs(vectorstore.similarity_search(out["query"], k=3))
).assign(prompt = lambda out: prompt_template.invoke({"context": out["context"],
                                                      "history": out["history"],
                                                     "query": out["query"]})
        ).assign(response = lambda out: llm.invoke(out["prompt"])
         ).assign(answer = answer_content)

In [58]:
my_lcel_try.invoke("What is Machine Learning?")

{'query': 'What is Machine Learning?',
 'history': [HumanMessage(content='What is Machine Learning?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Machine learning is a subset of artificial intelligence that allows systems to learn and improve from experience without being explicitly programmed. It involves training models on data to make predictions or decisions. There are three main types of machine learning: supervised, unsupervised, and reinforcement learning. Each type has its own approach to learning from data.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='can you give me your previous answer once again?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Sure! Here's the answer: \nMachine learning is a subset of artificial intelligence that allows systems to learn and improve from experience without being explicitly programmed. It involves training models on data to make predictions or decisions. There are three main 

In [69]:
import langchain_core

In [70]:
help(langchain_core)

Help on package langchain_core:

NAME
    langchain_core - `langchain-core` defines the base abstractions for the LangChain ecosystem.

DESCRIPTION
    The interfaces for core components like chat models, LLMs, vector stores, retrievers,
    and more are defined here. The universal invocation protocol (Runnables) along with
    a syntax for combining components (LangChain Expression Language) are also defined here.

    **No third-party integrations are defined here.** The dependencies are kept purposefully
    very lightweight.

PACKAGE CONTENTS
    _api (package)
    _import_utils
    agents
    caches
    callbacks (package)
    chat_history
    chat_loaders
    chat_sessions
    document_loaders (package)
    documents (package)
    embeddings (package)
    env
    example_selectors (package)
    exceptions
    globals
    indexing (package)
    language_models (package)
    load (package)
    messages (package)
    output_parsers (package)
    outputs (package)
    prompt_values
 

In [71]:
import langchain_core.chat_history

In [72]:
help(langchain_core.chat_history)

Help on module langchain_core.chat_history in langchain_core:

NAME
    langchain_core.chat_history - **Chat message history** stores a history of the message interactions in a chat.

CLASSES
    abc.ABC(builtins.object)
        BaseChatMessageHistory
            InMemoryChatMessageHistory(BaseChatMessageHistory, pydantic.main.BaseModel)

    class BaseChatMessageHistory(abc.ABC)
     |  Abstract base class for storing chat message history.
     |
     |  Implementations guidelines:
     |
     |  Implementations are expected to over-ride all or some of the following methods:
     |
     |  * add_messages: sync variant for bulk addition of messages
     |  * aadd_messages: async variant for bulk addition of messages
     |  * messages: sync variant for getting messages
     |  * aget_messages: async variant for getting messages
     |  * clear: sync variant for clearing messages
     |  * aclear: async variant for clearing messages
     |
     |  add_messages contains a default impleme

In [ ]:
import langchain

In [74]:
help(langchain)

Help on package langchain:

NAME
    langchain - Main entrypoint into LangChain.

PACKAGE CONTENTS
    agents (package)
    chat_models (package)
    embeddings (package)
    messages (package)
    rate_limiters (package)
    tools (package)

VERSION
    1.0.4

FILE
    c:\developer\rag_krishai_bootcamp\data_ingestion\.venv\lib\site-packages\langchain\__init__.py




#### Using GROQ LLM's

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_ollama import ChatOllama

In [95]:
ollama_llm = ChatOllama(model="gemma2:2b", temperature=0)
ollama_llm.invoke("What is Machine Learning?")

AIMessage(content='Machine learning (ML) is a subfield of artificial intelligence (AI) that focuses on enabling computers to learn from data without being explicitly programmed. \n\nHere\'s a breakdown:\n\n**How it works:**\n\n* **Data is the fuel:** ML algorithms are trained on large datasets, allowing them to identify patterns and relationships within the data.\n* **Algorithms do the learning:** These algorithms use mathematical models to analyze the data and make predictions or decisions based on what they learn. \n* **The more data, the better:** The quality and quantity of data directly impact the accuracy and performance of ML models.\n\n**Types of Machine Learning:**\n\n* **Supervised learning:**  Algorithms are trained on labeled datasets (e.g., images with labels like "cat" or "dog"). They learn to predict outcomes based on this labeled information.\n* **Unsupervised learning:** Algorithms analyze unlabeled data and try to find hidden patterns, structures, or relationships wit

In [ ]:
### A 2-b billion model took nearly ~ 40 seconds to respond in my local computer with 6 GB VRAM, 16GB RAM.

In [99]:
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model

In [106]:
import os

In [111]:
load_dotenv(override=True)

True

In [114]:
llm= ChatGroq(model="llama-3.1-8b-instant", temperature=0)
llm.invoke("What is Machine Learning?")

AIMessage(content='Machine learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time. The primary goal of machine learning is to enable machines to automatically improve their performance on a task without being explicitly programmed.\n\nMachine learning involves training a model on a dataset, which allows the model to learn patterns and relationships within the data. The model can then use this knowledge to make predictions, classify new data, or make decisions.\n\nThere are several key characteristics of machine learning:\n\n1. **Learning from data**: Machine learning models learn from data, rather than being explicitly programmed.\n2. **Improvement over time**: Machine learning models can improve their performance over time as they receive more data and experience.\n3. **Autonomy**: Machine learning models can make decisions and take ac

In [117]:
llm= ChatGroq(model="openai/gpt-oss-20b", temperature=0)
llm.invoke("What is Machine Learning?")

AIMessage(content='**Machine Learning (ML)** is a branch of artificial intelligence that enables computers to learn patterns and make decisions from data, without being explicitly programmed for each specific task.\n\n### Core Idea\n- **Learning from data**: Instead of hard‑coding rules, an ML system is fed examples (input–output pairs or raw observations) and it automatically discovers the underlying relationships.\n- **Generalization**: After training, the model can apply what it has learned to new, unseen data.\n\n### How It Works (Simplified)\n1. **Collect Data** – Gather a representative set of examples.\n2. **Choose a Model** – Pick an algorithm (e.g., linear regression, decision tree, neural network).\n3. **Train** – Feed the data to the model so it adjusts internal parameters to minimize error.\n4. **Validate** – Test on separate data to ensure it generalizes.\n5. **Deploy** – Use the trained model to make predictions or decisions in real‑world scenarios.\n\n### Types of Machin